# HD30501 Differential / new "handy" data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import Obtain_Telluric as obt
from Get_filenames import get_filenames
import time 
import datetime
from PyAstronomy import pyasl
%matplotlib inline

In [ ]:
# Import Bokeh modules for interactive plotting
import bokeh.io
import bokeh.mpl
import bokeh.plotting
%config InlineBackend.figure_formats = {'svg',}


In [ ]:
# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

In [ ]:
# Parameters to alter to change spectra seen
star = "HD30501"
chip_num = 1
obs_num = "1"
ref_num = "3"
use_h2o = False

target = "{}-{}".format(star, obs_num)
reference_target = "{}-{}".format(star, ref_num)    # should be different from target

if target == reference_target:
   raise ValueError("Reference target should be different from target")

In [ ]:
### Dracs data
dracs_path = "/home/jneal/Phd/data/Crires/BDs-DRACS/2017/{0}/Combined_Nods/".format(target)
#dracs_path = "../HD30501_data/{0}".format(obs_num)
dracs_name = get_filenames(dracs_path, "CRIRE.*","*{}.nod.ms.norm.mixavg.wavecal.fits".format(chip_num))
dracs_name = dracs_name[0]

In [ ]:
# Dracs data load

#dracs_data = fits.getdata(dracs_names[Chip_num-1])
#dracs_hdr = fits.getheader(dracs_names[Chip_num-1]) 
dracs_data = fits.getdata(dracs_name)
dracs_hdr = fits.getheader(dracs_name) 

dracs_wl = dracs_data["Wavelength"]
#dracs_I = dracs_data["Extracted_DRACS"]
dracs_I = dracs_data["Flux"]

# normalize dracs
maxes = dracs_I[(dracs_I < 1.2)].argsort()[-50:][::-1]
dracs_I = dracs_I / np.median(dracs_I[maxes])

chip_min_wl = dracs_wl[0]/1.0001
chip_max_wl = dracs_wl[-1]*1.0001

In [ ]:
# Load tapas file
tapas_path = dracs_path + "../Telluric_files/"
tapas_name = get_filenames(tapas_path, "tapas_*","*ReqId_10*")[0]

Tapas_data, Tapas_hdr = obt.load_telluric(tapas_path, tapas_name)
tell_wl = Tapas_data[0]
tell_I = Tapas_data[1]

# normalize dracs
maxes = tell_I[(tell_I < 1.2)].argsort()[-50:][::-1]
#tell_I = tell_I / np.median(tell_I[maxes])

#wl limit
wlmask = (tell_wl > dracs_wl[0]/1.0001) & (tell_wl < dracs_wl[-1]*1.0001)
tell_wl = tell_wl[wlmask]
tell_I = tell_I[wlmask] 

In [ ]:
# Corrected values
### handy data
handy_path = "/home/jneal/.handy_spectra/"
h2o = ["", "h2o"]
tellcorr_name = handy_path + "{0}-{1}-mixavg-{2}tellcorr_{3}.fits".format(star, obs_num, h2o[0], chip_num)
h20tellcorr_name = handy_path + "{0}-{1}-mixavg-{2}tellcorr_{3}.fits".format(star, obs_num, h2o[1], chip_num)

# tellcorr_name = get_filenames(dracs_path, "CRIRE.*","*{}.nod.ms.norm.sum.wavecal.tellcorr.fits".format(chip_num))
# h20tellcorr_name = get_filenames(dracs_path, "CRIRE.*","*{}.nod.ms.norm.sum.wavecal.h2otellcorr.fits".format(chip_num))
# print(tellcorr_name)
#tellcorr_name = dracs_path + tellcorr_name[0]
#h20tellcorr_name = dracs_path + h20tellcorr_name[0]

tellcorr_data = fits.getdata(tellcorr_name)
#print(tellcorr_data.columns)
tellcorr_hdr = fits.getheader(tellcorr_name) 
tellcorr_wl = tellcorr_data["Wavelength"]
# tellcorr_I = tellcorr_data["Corrected_DRACS"]
tellcorr_I = tellcorr_data["Flux"]
# tellcorr_tell = tellcorr_data["Interpolated_Tapas"]   # for masking

h20tellcorr_data = fits.getdata(h20tellcorr_name)
#print(h20tellcorr_data.columns)
h20tellcorr_hdr = fits.getheader(h20tellcorr_name) 
h20tellcorr_wl = h20tellcorr_data["Wavelength"]
#h20tellcorr_I = h20tellcorr_data["Corrected_DRACS"]
h20tellcorr_I = h20tellcorr_data["Flux"]

### Load Reference Target
Also Berv corrected

In [ ]:
### Reference data 
reftellcorr_name = handy_path + "{0}-{1}-mixavg-{2}tellcorr_{3}.fits".format(star, ref_num, h2o[0], chip_num)
refh20tellcorr_name = handy_path + "{0}-{1}-mixavg-{2}tellcorr_{3}.fits".format(star, ref_num, h2o[1], chip_num)

# tellcorr_name = get_filenames(dracs_path, "CRIRE.*","*{}.nod.ms.norm.sum.wavecal.tellcorr.fits".format(chip_num))
# h20tellcorr_name = get_filenames(dracs_path, "CRIRE.*","*{}.nod.ms.norm.sum.wavecal.h2otellcorr.fits".format(chip_num))
# print(tellcorr_name)
#tellcorr_name = dracs_path + tellcorr_name[0]
#h20tellcorr_name = dracs_path + h20tellcorr_name[0]

reftellcorr_data = fits.getdata(reftellcorr_name)
#print(tellcorr_data.columns)
reftellcorr_hdr = fits.getheader(reftellcorr_name) 
reftellcorr_wl = reftellcorr_data["Wavelength"]
# tellcorr_I = reftellcorr_data["Corrected_DRACS"]
reftellcorr_I = reftellcorr_data["Flux"]
# tellcorr_tell = reftellcorr_data["Interpolated_Tapas"]   # for masking

refh20tellcorr_data = fits.getdata(refh20tellcorr_name)
#print(h20tellcorr_data.columns)
refh20tellcorr_hdr = fits.getheader(refh20tellcorr_name) 
refh20tellcorr_wl = refh20tellcorr_data["Wavelength"]
# h20tellcorr_I = refh20tellcorr_data["Corrected_DRACS"]
refh20tellcorr_I = refh20tellcorr_data["Flux"]

In [ ]:
# Make barycorr function

def barycorr_CRIRES(wavelength, flux, header, extra_offset=None):
    #"""
    #Calculate Heliocenteric correction values and apply to spectrum.
   
    #SHOULD test again with bary and see what the  difference is.
    #"""

    longitude = float(header["HIERARCH ESO TEL GEOLON"])
    latitude = float(header["HIERARCH ESO TEL GEOLAT"])
    altitude = float(header["HIERARCH ESO TEL GEOELEV"])

    ra = header["RA"]    # CRIRES RA already in degrees 
    dec = header["DEC"]  # CRIRES hdr DEC already in degrees

    # Pyastronomy helcorr needs the time of observation in julian Days
    ##########################################################################################
    Time =  header["DATE-OBS"]    # Observing date  '2012-08-02T08:47:30.8425'
    # Get Average time **** from all raw files!!!  #################################################################

    wholetime, fractionaltime = Time.split(".")
    Time_time = time.strptime(wholetime, "%Y-%m-%dT%H:%M:%S")
    dt = datetime.datetime(*Time_time[:6])   # Turn into datetime object
    # Account for fractions of a second
    seconds_fractionalpart = float("0." + fractionaltime) / (24*60*60)   # Divide by seconds in a day

    # Including the fractional part of seconds changes pyasl.helcorr RV by the order of 1cm/s
    jd  = pyasl.asl.astroTimeLegacy.jdcnv(dt) + seconds_fractionalpart

    # Calculate helocentric velocity
    helcorr = pyasl.helcorr(longitude, latitude, altitude, ra, dec, jd, debug=False)
    
    if extra_offset:
        print("Warning!!!! have included a manual offset for testing")
        helcorr_val = helcorr[0] + extra_offset
    else:
        helcorr_val = helcorr[0]
    # Apply doopler shift to the target spectra with helcorr correction velocity 
    nflux, wlprime = pyasl.dopplerShift(wavelength, flux, helcorr_val, edgeHandling=None, fillValue=None)

    print(" RV size of heliocenter correction for spectra", helcorr_val)
    return nflux, wlprime

In [ ]:
manual_ofset_for_testing = 0
# Shift to same wavelength with barycentric correction
target_nflux, target_wlprime = barycorr_CRIRES(tellcorr_wl, tellcorr_I, tellcorr_hdr, extra_offset=manual_ofset_for_testing)

ref_nflux, ref_wlprime = barycorr_CRIRES(reftellcorr_wl, reftellcorr_I, reftellcorr_hdr)

# telluric line shift for masking
#tell_wl, tell_I
target_nflux_tell, __ = barycorr_CRIRES(tell_wl, tell_I, tellcorr_hdr)
ref_nfluxtell, __ = barycorr_CRIRES(tell_wl, tell_I, reftellcorr_hdr)

# Before and After Heliocentric Correction

In [ ]:
plt.plot(reftellcorr_wl, reftellcorr_I, label="Reference" )
plt.plot(tellcorr_wl, tellcorr_I, label="Target")

plt.title("Not BERV Corrected")
plt.xlabel("Wavelength(nm)")

bokeh.plotting.show(bokeh.mpl.to_bokeh())

In [ ]:
plt.plot(reftellcorr_wl, ref_nflux, label="Reference" )
plt.plot(tellcorr_wl, target_nflux, label="Target")

plt.title("BERV Corrected")
plt.xlabel("Wavelength(nm)")

bokeh.plotting.show(bokeh.mpl.to_bokeh())

In [ ]:

# wavelength reference untill spectral tools is fixed 
from scipy.interpolate import interp1d
def match_wl(wl, spec, ref_wl, method="scipy", kind="linear", bounds_error=False):
    """Interpolate Wavelengths of spectra to common WL
    Most likely convert telluric to observed spectra wl after wl mapping performed"""
    starttime = time.time()
    if method == "scipy":
        print(kind + " scipy interpolation")
        linear_interp = interp1d(wl, spec, kind=kind, bounds_error=False)
        new_spec = linear_interp(ref_wl)
    elif method == "numpy":
        if kind.lower() is not "linear":
            print("Warning: Cannot do " + kind + " interpolation with numpy, switching to linear" )
        print("Linear numpy interpolation")
        new_spec = np.interp(ref_wl, wl, spec)  # 1-d peicewise linear interpolat
    else:
        print("Method was given as " + method)
        raise("Not correct interpolation method specified")
    print("Interpolation Time = " + str(time.time() - starttime) + " seconds")

    return new_spec  # test inperpolations 


# Subtraction !

In [ ]:
# Shift to the reference wavelength scale for subtraction
### Old values
matched_tellcorr_I = match_wl(tellcorr_wl, target_nflux, reftellcorr_wl)

#subtracted_I = reftellcorr_I - matched_tellcorr_I    # O/C I think     ##### THIS was a BUG!!!

## BARY Corrected values

#target_nflux, target_wlprime = barycorr_CRIRES(tellcorr_wl, tellcorr_I, tellcorr_hdr, extra_offset=manual_ofset_for_testing)
#ref_nflux, ref_wlprime = barycorr_CRIRES(reftellcorr_wl, reftellcorr_I, reftellcorr_hdr)
#correct_match_I = match_wl(tellcorr_wl, target_nflux, reftellcorr_wl)

subtracted_I = ref_nflux - matched_tellcorr_I    ##### This fixed the bug and removed stellar lines very well!!!!
print(subtracted_I[6:-6])
print(matched_tellcorr_I)
print(ref_nflux)

plt.plot(reftellcorr_wl, subtracted_I)
plt.hlines(0, np.min(reftellcorr_wl), np.max(reftellcorr_wl), colors='k', linestyles='dashed', label='')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Recovered Difference")
plt.title("Difference between {0} and {1}".format(target, reference_target))
#bokeh.plotting.show(bokeh.mpl.to_bokeh())
plt.show()

In [ ]:
# Include masking
mask_limit = 0.95

print(subtracted_I)
from bokeh.plotting import figure, show, output_file, gridplot
from bokeh.layouts import column
from bokeh.models import BoxAnnotation

def bokeh_telluric_mask(fig, wl, I, mask_limit=0.9, fill_alpha=0.2, fill_color='red'):
    """ For use with bokeh"""
    wl_mask = I < mask_limit
    mean_step = np.mean([wl[1]-wl[0], wl[-1]-wl[-2]])   # Average nominal step size
    starts, ends = mask_edges(wl[wl_mask], mean_step)
    Boxes = [BoxAnnotation(plot=fig, left=start, right= end, fill_alpha=fill_alpha, fill_color=fill_color) for start, end in zip(starts, ends)]
    fig.renderers.extend(Boxes)
    
def matplotlib_telluric_mask(wl, I, mask_limit=0.9):
    """For use with matplotlib"""
    wl_mask = I < mask_limit
    mean_step = np.mean([wl[1]-wl[0], wl[-1]-wl[-2]])   # Average nominal step size
    starts, ends = mask_edges(wl[wl_mask], mean_step)
    [plt.axvspan(start, end, facecolor='g', alpha=0.5) for start, end in zip(starts, ends)] 
    
def mask_edges(wl, mean_step):
    beginings = [wav2 for wav1, wav2 in zip(wl[:-1], wl[1:]) if wav2-wav1 > 3*np.abs(mean_step)]
    ends = [wav1 for wav1, wav2 in zip(wl[:-1], wl[1:]) if wav2-wav1 > 3*np.abs(mean_step)]
    
    # prepend start of first line, and append end of last line
    beginings = [wl[0]] + beginings   # prepend starting value
    ends = ends + [wl[-1]] # append end value
    
    return beginings, ends

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(tools=TOOLS)

p.line(reftellcorr_wl, subtracted_I)
#plt.hlines(0, np.min(reftellcorr_wl), np.max(reftellcorr_wl), colors='k', linestyles='dashed', label='')

bokeh_telluric_mask(p, tell_wl, ref_nfluxtell, mask_limit=mask_limit, fill_alpha=0.4, fill_color='green')
bokeh_telluric_mask(p, reftellcorr_wl, reftellcorr_I, mask_limit=mask_limit, fill_alpha=0.4, fill_color='yellow')
p.title.text = "Comparison with Masks"
p.xaxis.axis_label = 'Wavelength'
p.yaxis.axis_label = 'Signal'
show(p)



In [ ]:
# Combine all 3 together
from bokeh.models import Range1d
# Following example from http://bokeh.pydata.org/en/latest/docs/user_guide/quickstart.html
fig_height = 250
fig_width = 800

s1 = figure(width=fig_width, height=fig_height, title="HD30501 Spectrum with telluric line model")
s1.line([np.min(tellcorr_wl), np.max(tellcorr_wl)], [1,1], color="black", line_dash="dashed", line_width=1)
s1.line(tell_wl, tell_I, legend="TAPAS", color="blue", line_width=2)
bokeh_telluric_mask(s1, tell_wl, tell_I, mask_limit=mask_limit, fill_alpha=0.4, fill_color='green')
bokeh_telluric_mask(s1, tellcorr_wl, tellcorr_I, mask_limit=mask_limit, fill_alpha=0.4, fill_color='yellow')
#s1.line(gas_wl, gas_I, legend="ESO", color="blue", line_dash="dotdash", line_width=2)
s1.line(dracs_wl, dracs_I, legend="HD30501", color="red", line_dash="dashed",line_width=2)

#plt.plot(gas_wl, gas_I, label="Gasgano")
#plt.plot(dracs_wl, dracs_I, label="Dracs")
#plt.plot(tell_wl, tell_I, label="Tapas")
#s1.title.text = "HD30501 Spectrum"
s1.xaxis.axis_label = 'Wavelength (nm)'
s1.yaxis.axis_label = 'Nomalized Intensity'
s1.legend.location = "bottom_right"
s1.title.text_font_size = "12pt"
s1.xaxis.axis_label_text_font_size = "12pt"
s1.yaxis.axis_label_text_font_size = "12pt"
s1.legend.border_line_color = None
s1.x_range = Range1d(chip_min_wl, chip_max_wl)
s1.y_range = Range1d(0.8, 1.03)
# s1.set(x_range=Range1d(chip_min_wl, chip_max_wl), y_range=Range1d(0.8, 1.03))  #Edit wl range

# NEW: Tapas normal and H20 Scaling
s2 = figure(width=fig_width, height=fig_height, x_range=s1.x_range, y_range=s1.y_range, 
            title="Telluric correction through division of the telluric line model")
s2.line([np.min(tellcorr_wl), np.max(tellcorr_wl)], [1,1], color="black", line_dash="dashed", line_width=1)
#s2.line(tellcorr_wl, tellcorr_I, legend="Airmass Scaling", color="blue", line_width=2)
#s2.line(h20tellcorr_wl, h20tellcorr_I, legend="H20 Scaling", color="red", line_dash="dashed", line_width=2)
s2.line(tellcorr_wl, tellcorr_I, legend="", color="blue", line_dash="solid", line_width=2)
 
bokeh_telluric_mask(s2, tell_wl, tell_I, mask_limit=mask_limit, fill_alpha=0.4, fill_color='green')
bokeh_telluric_mask(s2, tellcorr_wl, tellcorr_I, mask_limit=mask_limit, fill_alpha=0.4, fill_color='yellow')

#s2.title.text = "Telluric correction by division of telluric line model"
s2.title.text_font_size = "12pt"
s2.xaxis.axis_label = 'Wavelength (nm)'
s2.xaxis.axis_label_text_font_size = "12pt"
s2.yaxis.axis_label = 'Normalized Intensity'
s2.yaxis.axis_label_text_font_size = "12pt"
s2.legend.location = None
#s2.legend.location = "bottom_right"
#s2.legend.border_line_color = None
#plt.xlabel("Wavelength(nm)")

# NEW: create a new plot and share only one range
s3 = figure(width=fig_width, height=fig_height-40, x_range=s1.x_range)
s3.line([np.min(reftellcorr_wl), np.max(reftellcorr_wl)], [0,0], color="black", line_dash="dashed", line_width=1)
s3.line(reftellcorr_wl, subtracted_I, color="blue", line_width=2)
bokeh_telluric_mask(s3, tell_wl, ref_nfluxtell, mask_limit=mask_limit, fill_alpha=0.4, fill_color='green')
bokeh_telluric_mask(s3, reftellcorr_wl, reftellcorr_I, mask_limit=mask_limit, fill_alpha=0.4, fill_color='yellow')

s3.title.text = "Subtraction of two barycentic RV corrected observations"
s3.title.text_font_size = "12pt"
s3.xaxis.axis_label = 'Wavelength (nm)'
s3.xaxis.axis_label_text_font_size = "12pt"
s3.yaxis.axis_label = 'Difference'
s3.yaxis.axis_label_text_font_size = "12pt"
s3.legend.location = "bottom_right"
s3.legend.border_line_color = None
#p = gridplot([[s1],[s2],[s3]], toolbar_location=None)

# show the results
#show(p)

show(column(s1, s2, s3))



# Add RV difference of HD30501 b to TAPAS spectra. 


In [ ]:
Scaling_Factor = 0.004 # F/Fstar ~ 1/250  Incorrect value
Scaling_Factor = 0.012 # F/Fstar ~ 1/86  Better value


# Load tapas file
tapas_path = dracs_path + "../Telluric_files/"
tapas_name = get_filenames(tapas_path, "tapas_*","*ReqId_10*")[0]

Tapas_data, Tapas_hdr = obt.load_telluric(tapas_path, tapas_name)
tell_wl = Tapas_data[0]
tell_I = Tapas_data[1]

# Normalize dracs
maxes = tell_I[(tell_I < 1.2)].argsort()[-50:][::-1]
#tell_I = tell_I / np.median(tell_I[maxes])

#wl limit
#wlmask = (tell_wl > dracs_wl[0]/1.0001) & (tell_wl < dracs_wl[-1]*1.0001)
#tell_wl = tell_wl[wlmask]
#tell_I = tell_I[wlmask] 

In [ ]:
#Calculated Planet RV 
RV_planet = {"1":2327.66695686, "2a":1152.7794576, "2b":1142.77278133,"3":1101.47665896} #m/s
#obs_num = "1"
#ref_num = "4"
#Doppler shift each telluric spectrum by the corresponding values
nflux_rvobs, wlprime_rvobs = pyasl.dopplerShift(tell_wl, tell_I, RV_planet[obs_num]/1000, edgeHandling=None, fillValue=None)
nflux_rvref, wlprime_rvref = pyasl.dopplerShift(tell_wl, tell_I, RV_planet[ref_num]/1000, edgeHandling=None, fillValue=None)


tapas_diff = nflux_rvobs - nflux_rvref
print(tapas_diff)


In [ ]:
s4 = figure(width=fig_width, height=fig_height-40, x_range=s1.x_range)
s4.line([np.min(reftellcorr_wl), np.max(reftellcorr_wl)], [0,0], color="black", line_dash="dashed", line_width=1)
s4.line(tell_wl, tapas_diff , color="blue", line_width=2)
#bokeh_telluric_mask(s4, reftellcorr_wl, ref_nfluxtell, mask_limit=0.95, fill_alpha=0.4, fill_color='green')
#bokeh_telluric_mask(s4, reftellcorr_wl, reftellcorr_I, mask_limit=0.95, fill_alpha=0.4, fill_color='yellow')

s4.title.text = "Subtraction of Two Tapas spectra with RV offset of obs"
s4.title.text_font_size = "12pt"
s4.xaxis.axis_label = 'Wavelength (nm)'
s4.xaxis.axis_label_text_font_size = "12pt"
s4.yaxis.axis_label = 'Difference'
s4.yaxis.axis_label_text_font_size = "12pt"
s4.legend.location = "bottom_right"
s4.legend.border_line_color = None
#p = gridplot([[s1],[s2],[s3]], toolbar_location=None)

# Scale flux of spectrum by F/Fstar
scaled_tapas_diff = tapas_diff * Scaling_Factor

s5 = figure(width=fig_width, height=fig_height-40, x_range=s1.x_range)
s5.line([np.min(reftellcorr_wl), np.max(reftellcorr_wl)], [0,0], color="black", line_dash="dashed", line_width=1)
s5.line(tell_wl, scaled_tapas_diff, color="blue", line_width=2)
#bokeh_telluric_mask(s5, reftellcorr_wl, ref_nfluxtell, mask_limit=0.95, fill_alpha=0.4, fill_color='green')
#bokeh_telluric_mask(s5, reftellcorr_wl, reftellcorr_I, mask_limit=0.95, fill_alpha=0.4, fill_color='yellow')

s5.title.text = "Subtraction of Two Tapas spectra with RV offset of obs Scaled by F/Fstar"
s5.title.text_font_size = "12pt"
s5.xaxis.axis_label = 'Wavelength (nm)'
s5.xaxis.axis_label_text_font_size = "12pt"
s5.yaxis.axis_label = 'Difference'
s5.yaxis.axis_label_text_font_size = "12pt"
s5.legend.location = "bottom_right"
s5.legend.border_line_color = None
#p = gridplot([[s1],[s2],[s3]], toolbar_location=None)
s5.y_range = Range1d(-0.02, 0.02)  #Edit wl range
# show the results
#show(p)
s3.y_range = Range1d(-0.02, 0.02)  #Edit wl range
show(column(s4, s5, s3))

#### 

## Simulate BD as the Stellar spectrum for the RV values
The result with using tellruic correction is not recommended
Pedro suggested adding a smaller version of star spectrum to stellar spectrum and then subtract them

In [ ]:
Star_wl = tellcorr_wl
Star_I = target_nflux
ref_Star_wl = reftellcorr_wl
ref_Star_I = ref_nflux
F_ratio = Scaling_Factor

print(RV_planet)
starflux_rvobs, starwlprime_rvobs = pyasl.dopplerShift(Star_wl, Star_I, RV_planet[obs_num]/1000, edgeHandling=None, fillValue=None)
starflux_rvobs2, starwlprime_rvobs2 = pyasl.dopplerShift(Star_wl, Star_I, RV_planet[ref_num]/1000, edgeHandling=None, fillValue=None)
starflux_rvref, starwlprime_rvref = pyasl.dopplerShift(ref_Star_wl, ref_Star_I, RV_planet[ref_num]/1000, edgeHandling=None, fillValue=None)

Star_obs = Star_I + starflux_rvobs * F_ratio
Star_obs2 = Star_I + starflux_rvobs2 * F_ratio
Star_ref = ref_Star_I + starflux_rvref * F_ratio


# match wl of ref
wl_matched_Star_I = match_wl(Star_wl, Star_obs, ref_Star_wl)
Simulation_subtracted_I = Star_ref - wl_matched_Star_I    ##### This fixed the bug and removed stellar lines very well!!!!

Subtract_original_residuals = Simulation_subtracted_I - subtracted_I
print(subtracted_I[6:-6])
print(Subtract_original_residuals[6:-6])

In [ ]:
# Plot the RV shifted spectra

ss = figure(width=fig_width, height=fig_height-40, x_range=s1.x_range)
ss.line([np.min(reftellcorr_wl), np.max(reftellcorr_wl)], [1,1], color="black", line_dash="dashed", line_width=1)
ss.line(Star_wl, Star_obs , color="blue", line_width=1)
ss.line(Star_wl, Star_obs2 , color="red", line_width=1)
print("RV difference", RV_planet[ref_num]/1000 - RV_planet[obs_num]/1000, "km/s")
print("wavelength  difference", (RV_planet[ref_num]/1000 - RV_planet[obs_num]/1000)/3e5 * 2000, "nm")

show(ss)

In [ ]:
s6 = figure(width=fig_width, height=fig_height-40, x_range=s1.x_range)
s6.line([np.min(reftellcorr_wl), np.max(reftellcorr_wl)], [1,1], color="black", line_dash="dashed", line_width=1)
s6.line(Star_wl, Star_obs , color="blue", line_width=1)
s6.line(Star_wl, Star_ref , color="red", line_width=1)
#bokeh_telluric_mask(s4, reftellcorr_wl, ref_nfluxtell, mask_limit=0.95, fill_alpha=0.4, fill_color='green')
#bokeh_telluric_mask(s4, reftellcorr_wl, reftellcorr_I, mask_limit=0.95, fill_alpha=0.4, fill_color='yellow')

s6.title.text = "Stellar spectra with added BD spectra at RV offset of obs (I(lambda) + F_star*I(lambda+RV))"
s6.title.text_font_size = "12pt"
s6.xaxis.axis_label = 'Wavelength (nm)'
s6.xaxis.axis_label_text_font_size = "12pt"
s6.yaxis.axis_label = 'Flux'
s6.yaxis.axis_label_text_font_size = "12pt"
s6.legend.location = "bottom_right"
s6.legend.border_line_color = None
#p = gridplot([[s1],[s2],[s3]], toolbar_location=None)


s7 = figure(width=fig_width, height=fig_height-40, x_range=s1.x_range)
s7.line([np.min(reftellcorr_wl), np.max(reftellcorr_wl)], [0,0], color="black", line_dash="dashed", line_width=1)
#s7.line(ref_Star_wl, Simulation_subtracted_I, color="blue", line_width=2)
#s7.line(ref_Star_wl, Subtract_original_residuals,  color="blue", line_width=2)
s7.line(ref_Star_wl, Simulation_subtracted_I, legend="Residual", color="blue", line_width=1)
s7.line(ref_Star_wl, Subtract_original_residuals, legend="Effect of adding planet", color="red", line_width=1)
#s7.line(Star_wl, Star_ref , color="red", line_width=2)
bokeh_telluric_mask(s7, reftellcorr_wl, ref_nfluxtell, mask_limit=0.95, fill_alpha=0.4, fill_color='green')
bokeh_telluric_mask(s7, reftellcorr_wl, reftellcorr_I, mask_limit=0.95, fill_alpha=0.4, fill_color='yellow')

#s7.title.text = "Difference Stellar spectra with added BD spectra at RV offset of obs (I(lambda) + F_star*I(lambda+RV))"
s7.title.text = "BD with stellar spectra at the calculated RV offsets"
s7.title.text_font_size = "12pt"
s7.xaxis.axis_label = 'Wavelength (nm)'
s7.xaxis.axis_label_text_font_size = "12pt"
s7.yaxis.axis_label = 'Difference'
s7.yaxis.axis_label_text_font_size = "12pt"
s7.legend.location = "bottom_right"
s7.legend.border_line_color = None
#p = gridplot([[s1],[s2],[s3]], toolbar_location=None)

print()

In [ ]:
show(column(s6, s7))

In [ ]:
s8 = figure(width=fig_width, height=fig_height-40, x_range=s1.x_range)
s8.line([np.min(reftellcorr_wl), np.max(reftellcorr_wl)], [0,0], color="black", line_dash="dashed", line_width=1)
#s8.line(Star_wl, Star_obs - Star_ref, color="blue", line_width=2)
s8.line(Star_wl, Star_ref, color="red", line_width=2)
s8.line(Star_wl, Star_obs, color="green", line_width=2)
#bokeh_telluric_mask(s4, reftellcorr_wl, ref_nfluxtell, mask_limit=0.95, fill_alpha=0.4, fill_color='green')
#bokeh_telluric_mask(s4, reftellcorr_wl, reftellcorr_I, mask_limit=0.95, fill_alpha=0.4, fill_color='yellow')

s8.title.text = "Stellar spectra with added BD spectra at RV offset of obs (I(lambda) + F_star*I(lambda+RV))"
s8.title.text_font_size = "12pt"
s8.xaxis.axis_label = 'Wavelength (nm)'
s8.xaxis.axis_label_text_font_size = "12pt"
s8.yaxis.axis_label = 'Difference'
s8.yaxis.axis_label_text_font_size = "12pt"
s8.legend.location = "bottom_right"
s8.legend.border_line_color = None
#p = gridplot([[s1],[s2],[s3]], toolbar_location=None)
s3.y_range=Range1d(-0.5, 0.5)  #Edit wl range
#s8.y_range=Range1d(-0.5, 0.5)  #Edit wl range
show(column(s3, s8))


# Signal to Noise

In [ ]:
# Standard deviation of residuals after subtraction
std1 = np.nanstd(subtracted_I)
print("Standard deviation of differnce spectra", std1)
print("S/N = 1/sigma = ", 1/std1)

# Select only 2114- 2116 nm
mask = (reftellcorr_wl > 2112) & (reftellcorr_wl < 2118)

w = reftellcorr_wl[mask]
sub_i = subtracted_I[mask]

std2 = np.std(sub_i)
print("2114-2116 std", std2)
print("S/N 2114-2116 std", 1/std2)

print("Max val", np.max(sub_i))
print("Min val", np.min(sub_i))


In [ ]:
rands = np.random.random(1000)*0.04 - 0.02
a = np.std(rands)
print("S/N of random values", 1/a)

# Simulate Maxium shift of BD orbit
# Added June 2017

In [ ]:
# For HD30501 the K2 value is
rv_k2 = 19.8  # Semi-ampliture
rv_max = 2 * rv_k2    # Maximum RV separation possible.

Star_wl = tellcorr_wl
Star_I = target_nflux
ref_Star_wl = reftellcorr_wl
ref_Star_I = ref_nflux
F_ratio = Scaling_Factor
print("F_ratio is", F_ratio)

# Maxium RV of HD30501
print(RV_planet)
starflux_rvobs, starwlprime_rvobs = pyasl.dopplerShift(Star_wl, Star_I, RV_planet[obs_num]/1000, edgeHandling=None, fillValue=None)
starflux_rvobs_k2, starwlprime_rvobs_k2 = pyasl.dopplerShift(Star_wl, Star_I, rv_k2, edgeHandling=None, fillValue=None)
starflux_rvobs2_maxrv, starwlprime_rvobs2 = pyasl.dopplerShift(Star_wl, Star_I, rv_max , edgeHandling=None, fillValue=None)
starflux_rvref, starwlprime_rvref = pyasl.dopplerShift(ref_Star_wl, ref_Star_I, 0, edgeHandling=None, fillValue=None)

Star_obs_obs = (Star_I + starflux_rvobs * F_ratio) / (1 + F_ratio)
Star_obs = (Star_I + starflux_rvobs_k2 * F_ratio) / (1 + F_ratio)
Star_obs2 = (Star_I + starflux_rvobs2_maxrv * F_ratio) / (1 + F_ratio)
Star_ref = (ref_Star_I + starflux_rvref * F_ratio) / (1 + F_ratio)


# Match wl of ref
wl_matched_Star_I_obs = match_wl(Star_wl, Star_obs_obs, ref_Star_wl)
wl_matched_Star_I = match_wl(Star_wl, Star_obs, ref_Star_wl)
wl_matched_Star_I2 = match_wl(Star_wl, Star_obs2, ref_Star_wl)

Simulation_subtracted_I_obsrv = Star_ref - wl_matched_Star_I_obs
Simulation_subtracted_I = Star_ref - wl_matched_Star_I    ##### This fixed the bug and removed stellar lines very well!!!!
Simulation_subtracted_I2 = Star_ref - wl_matched_Star_I2  

Subtract_original_residuals = Simulation_subtracted_I - subtracted_I
Subtract_original_residuals2 = Simulation_subtracted_I2 - subtracted_I

print("k2 subI", subtracted_I[6:-6])
print("k2 residuals", Subtract_original_residuals[6:-6])
print("max_rv subI", subtracted_I[6:-6])
print("max rv residuals", Subtract_original_residuals2[6:-6])


In [ ]:
# Plot the RV shifted spectra

ss = figure(width=fig_width, height=fig_height-40, x_range=s1.x_range)
ss.line([np.min(reftellcorr_wl), np.max(reftellcorr_wl)], [1,1], color="black", line_dash="dashed", line_width=1)
ss.line(Star_wl, Star_I , color="blue", line_width=1, legend="Original")
ss.line(Star_wl, Star_obs , color="red", line_width=1, legend="With simulated planet at K2.")
ss.line(Star_wl, Star_obs2 , color="green", line_width=1, legend="With simulated planet at max rv")
ss.line(Star_wl, starflux_rvobs2_maxrv , color="pink", line_width=1, legend="With simulated planet at max rv")
print("RV difference", RV_planet[ref_num]/1000 - RV_planet[obs_num]/1000, "km/s")
print("wavelength  difference", (RV_planet[ref_num]/1000 - RV_planet[obs_num]/1000)/3e5 * 2000, "nm")

show(ss)

In [ ]:
s6 = figure(width=fig_width, height=fig_height-40, x_range=s1.x_range)
s6.line([np.min(reftellcorr_wl), np.max(reftellcorr_wl)], [1,1], color="black", line_dash="dashed", line_width=1)
s6.line(Star_wl, Star_obs , color="blue", line_width=1)
s6.line(Star_wl, Star_ref , color="red", line_width=1)
#bokeh_telluric_mask(s4, reftellcorr_wl, ref_nfluxtell, mask_limit=0.95, fill_alpha=0.4, fill_color='green')
#bokeh_telluric_mask(s4, reftellcorr_wl, reftellcorr_I, mask_limit=0.95, fill_alpha=0.4, fill_color='yellow')

s6.title.text = "Stellar spectra with added BD spectra at RV offset of obs (I(wl) + F_ratio * I(wl+RV))"
s6.title.text_font_size = "12pt"
s6.xaxis.axis_label = 'Wavelength (nm)'
s6.xaxis.axis_label_text_font_size = "12pt"
s6.yaxis.axis_label = 'Flux'
s6.yaxis.axis_label_text_font_size = "12pt"
s6.legend.location = "bottom_right"
s6.legend.border_line_color = None
#p = gridplot([[s1],[s2],[s3]], toolbar_location=None)


s7 = figure(width=fig_width, height=fig_height-40, x_range=s1.x_range)
s7.line([np.min(reftellcorr_wl), np.max(reftellcorr_wl)], [0,0], color="black", line_dash="dashed", line_width=1)
#s7.line(ref_Star_wl, Simulation_subtracted_I, color="blue", line_width=2)
#s7.line(ref_Star_wl, Subtract_original_residuals,  color="blue", line_width=2)
s7.line(ref_Star_wl, Simulation_subtracted_I, legend="Residual", color="blue", line_width=1)
s7.line(ref_Star_wl, Subtract_original_residuals, legend="Effect of adding planet at k2", color="red", line_width=1)
s7.line(ref_Star_wl, Subtract_original_residuals2, legend="Effect of adding planet at max rv", color="green", line_width=1)
#s7.line(Star_wl, Star_ref , color="red", line_width=2)
bokeh_telluric_mask(s7, reftellcorr_wl, ref_nfluxtell, mask_limit=0.95, fill_alpha=0.4, fill_color='green')
bokeh_telluric_mask(s7, reftellcorr_wl, reftellcorr_I, mask_limit=0.95, fill_alpha=0.4, fill_color='yellow')

#s7.title.text = "Difference Stellar spectra with added BD spectra at RV offset of obs (I(lambda) + F_star*I(lambda+RV))"
s7.title.text = "BD with stellar spectra at the calculated RV offsets"
s7.title.text_font_size = "12pt"
s7.xaxis.axis_label = 'Wavelength (nm)'
s7.xaxis.axis_label_text_font_size = "12pt"
s7.yaxis.axis_label = 'Difference'
s7.yaxis.axis_label_text_font_size = "12pt"
s7.legend.location = "bottom_right"
s7.legend.border_line_color = None
#p = gridplot([[s1],[s2],[s3]], toolbar_location=None)

print()


In [ ]:
show(column(s6, s7))

In [ ]:
s8 = figure(width=fig_width, height=fig_height-40, x_range=s1.x_range)
s8.line([np.min(reftellcorr_wl), np.max(reftellcorr_wl)], [0,0], color="black", line_dash="dashed", line_width=1)
#s8.line(Star_wl, Star_obs - Star_ref, color="blue", line_width=2)
s8.line(Star_wl, Star_ref, color="red", line_width=2)
s8.line(Star_wl, Star_obs, color="green", line_width=2)
#bokeh_telluric_mask(s4, reftellcorr_wl, ref_nfluxtell, mask_limit=0.95, fill_alpha=0.4, fill_color='green')
#bokeh_telluric_mask(s4, reftellcorr_wl, reftellcorr_I, mask_limit=0.95, fill_alpha=0.4, fill_color='yellow')

s8.title.text = "Stellar spectra with added BD spectra at RV offset of obs (I(lambda) + F_star*I(lambda+RV))"
s8.title.text_font_size = "12pt"
s8.xaxis.axis_label = 'Wavelength (nm)'
s8.xaxis.axis_label_text_font_size = "12pt"
s8.yaxis.axis_label = 'Difference'
s8.yaxis.axis_label_text_font_size = "12pt"
s8.legend.location = "bottom_right"
s8.legend.border_line_color = None
#p = gridplot([[s1],[s2],[s3]], toolbar_location=None)
s3.y_range=Range1d(-0.5, 0.5)  #Edit wl range
#s8.y_range=Range1d(-0.5, 0.5)  #Edit wl range
show(column(s3, s8))

In [ ]:

# RESIDUAL CHECKS 

# ADD THE SPECTA TO ITSELF once at 0 and once shifted.


#The differences in 

s11 = figure(width=fig_width, height=fig_height+50, x_range=s1.x_range)
s11.line(ref_Star_wl, Simulation_subtracted_I_obsrv, legend="RV=2.3 km/s", color="blue")
s11.line(ref_Star_wl, Simulation_subtracted_I, legend="RV=19.8 km/s", color="black",)
s11.line(ref_Star_wl, Simulation_subtracted_I2, legend="RV=39.6 km/s",color="red")

s11.title.text = "Simulated RV difference"
s11.title.text_font_size = "12pt"
s11.xaxis.axis_label = 'Wavelength (nm)'
s11.xaxis.axis_label_text_font_size = "12pt"
s11.yaxis.axis_label = 'Flux Difference'
s11.yaxis.axis_label_text_font_size = "12pt"
s11.legend.location = "bottom_right"
s11.legend.border_line_color = None



s12 = figure(width=fig_width, height=fig_height+50, x_range=s1.x_range)
s12.line(ref_Star_wl, Simulation_subtracted_I_obsrv, legend="RV=2.3 km/s")
s12.line(ref_Star_wl, Simulation_subtracted_I - Simulation_subtracted_I_obsrv, color="black", legend="RV=19.8 km/s", line_width=2)
s12.line(ref_Star_wl, Simulation_subtracted_I2 - Simulation_subtracted_I_obsrv, color="red", legend="RV=39.6 km/s", line_width=2)

s12.title.text = "Simulated RV residual difference"
s12.title.text_font_size = "12pt"
s12.xaxis.axis_label = 'Wavelength (nm)'
s12.xaxis.axis_label_text_font_size = "12pt"
s12.yaxis.axis_label = 'Flux Difference'
s12.yaxis.axis_label_text_font_size = "12pt"
s12.legend.location = "bottom_right"
s12.legend.border_line_color = None

show(column(s11, s12))



In [ ]:

# Shift itself by 2kms and subtract:
    
# Star_I

starflux_rvobs, starwlprime_rvobs = pyasl.dopplerShift(Star_wl, Star_I, RV_planet[obs_num]/1000, edgeHandling=None, fillValue=None)
starflux_rvobs_k2, starwlprime_rvobs_k2 = pyasl.dopplerShift(Star_wl, Star_I, rv_k2, edgeHandling=None, fillValue=None)
starflux_rvobs2_maxrv, starwlprime_rvobs2 = pyasl.dopplerShift(Star_wl, Star_I, rv_max , edgeHandling=None, fillValue=None)
starflux_rvref = Star_I

Star_obs_obs = starflux_rvobs
Star_obs = starflux_rvobs_k2 
Star_obs2 =starflux_rvobs2_maxrv
Star_ref = starflux_rvref


# Match wl of ref
wl_matched_Star_I_obs = match_wl(Star_wl, Star_obs_obs, ref_Star_wl)
wl_matched_Star_I = match_wl(Star_wl, Star_obs, ref_Star_wl)
wl_matched_Star_I2 = match_wl(Star_wl, Star_obs2, ref_Star_wl)

Subtracted_I_obsrv = Star_ref - wl_matched_Star_I_obs
Subtracted_I_k2 = Star_ref - wl_matched_Star_I    ##### This fixed the bug and removed stellar lines very well!!!!
Subtracted_I2_maxrv = Star_ref - wl_matched_Star_I2  

Subtract_original_residuals = Simulation_subtracted_I - subtracted_I
Subtract_original_residuals2 = Simulation_subtracted_I2 - subtracted_I


In [ ]:
# Plot the spectrum


s10 = figure(width=fig_width, height=fig_height+50, x_range=s1.x_range)
s10.line(ref_Star_wl, Subtracted_I_obsrv + 0.4, legend="RV=2.3 km/s", color="blue", line_width=2)
s10.line(ref_Star_wl, Subtracted_I_k2 + 0.2, legend="RV=19.8 km/s", color="black", line_width=2)
s10.line(ref_Star_wl, Subtracted_I2_maxrv + 0, legend="RV=39.6 km/s", color="red", line_width=2)

s10.title.text = "Host Star RV difference"
s10.title.text_font_size = "12pt"
s10.xaxis.axis_label = 'Wavelength (nm)'
s10.xaxis.axis_label_text_font_size = "12pt"
s10.yaxis.axis_label = 'Flux Difference'
s10.yaxis.axis_label_text_font_size = "12pt"
s10.legend.location = "bottom_right"
s10.legend.border_line_color = None

show(column(s10))


## Plot Simulated RV difference on the measured difference 
Copied here for 2017 progress report

In [ ]:

import copy
s11 = copy.copy(s3)
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import PyAstronomy.pyasl as pyasl
pathwave = "/home/jneal/Phd/data/phoenixmodels/WAVE_PHOENIX-ACES-AGSS-COND-2011.fits"
specpath = "/home/jneal/Phd/data/phoenixmodels/HD30501b-lte02500-5.00-0.0.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits"
w_mod = fits.getdata(pathwave)
w_mod /= 10   # turn into nm
flux = fits.getdata(specpath)
hdr = fits.getheader(specpath)
# Cut at 2050 - 2150nm
wtmp = w_mod
f_tmp = flux
mask = (w_mod < 2123) & (w_mod > 2011)
w_mod = w_mod[mask]
flux = flux[mask]
# Convove to 50000
flux = pyasl.instrBroadGaussFast(w_mod, flux, 50000, edgeHandling=None, fullout=False, maxsig=None)
# Normalize
flux /= np.median(flux)
maxes = flux.argsort()[-100:][::-1]
flux /= np.median(flux[maxes])
flux[flux < 0] = 0
# Calculated Planet RV 
RV_planet = {"1":2327.66695686, "2a":1152.7794576, "2b":1142.77278133,"3":1101.47665896} #m/s
obs_num = "1"
ref_num = "3"
nflux_rvobs, wlprime_rvobs = pyasl.dopplerShift(w_mod, flux, RV_planet[obs_num]/1000, edgeHandling=None, fillValue=None)
nflux_rvref, wlprime_rvref = pyasl.dopplerShift(w_mod, flux, RV_planet[ref_num]/1000, edgeHandling=None, fillValue=None)
spec_diff = nflux_rvobs - nflux_rvref

s11.line(w_mod, 0.01 * spec_diff, color="red", line_dash="solid", line_width=1)



fig_height = 500
fig_width = 800

#s11.width = fig_width
s11.height = fig_height
#s11.title.text = "Blash"

In [ ]:
fig_height = 400
fig_width = 800

#s11.width = fig_width
s11.height = fig_height
s11.x_range = Range1d(2111.9, 2123)
s11.y_range = Range1d(-0.021, 0.021)
show(s11)

In [ ]:
## CROSS-CORRELATE !!!!
#wspec = w_tmp
#fspec = 0.01 * spec_diff

#wtmp = w_mod
#f_tmp = f_mod
flux = f_tmp
w_mod = wtmp

# MASK
mask = (w_mod < 2143) & (w_mod > 2101)
w_mod = w_mod[mask]
flux = flux[mask]

flux /= np.median(flux)
maxes = flux.argsort()[-100:][::-1]
flux /= np.median(flux[maxes])
flux[flux < 0] = 0
# Calculated Planet RV 
RV_planet = {"1":2327.66695686, "2a":1152.7794576, "2b":1142.77278133,"3":1101.47665896} #m/s
obs_num = "1"
ref_num = "3"
nflux_rvobs, wlprime_rvobs = pyasl.dopplerShift(w_mod, flux, RV_planet[obs_num]/1000, edgeHandling=None, fillValue=None)
nflux_rvref, wlprime_rvref = pyasl.dopplerShift(w_mod, flux, RV_planet[ref_num]/1000, edgeHandling=None, fillValue=None)
spec_diff = nflux_rvobs - nflux_rvref



wobs = reftellcorr_wl
fobs = subtracted_I



In [ ]:

# if np.any(np.isnan(fobs)):
#    print("isnan")
x = pyasl.crosscorrRV(wobs[~np.isnan(fobs)], fobs[~np.isnan(fobs)], w_mod, spec_diff, rvmin=-150, rvmax=150, drv=0.1, mode="lin", skipedge=True)

y = pyasl.crosscorrRV(wobs[~np.isnan(fobs)], fobs[~np.isnan(fobs)], w_mod, spec_diff, rvmin=-6, rvmax=6, drv=0.01, mode="lin", skipedge=True)



In [ ]:
# print(x)
plt.plot(x[0], x[1])
plt.show()

plt.plot(y[0], y[1])
plt.show()

In [ ]:
plt.plot(w_mod, spec_diff*0.01)
plt.plot(wobs, fobs)

plt.show()

In [ ]:
# Auto correlation of template
mask = (w_mod < 2133) & (w_mod > 2111)
w_mod2 = w_mod[mask]
flux2 = flux[mask]
spec_diff2 = spec_diff[mask]

z = pyasl.crosscorrRV(w_mod2, spec_diff2, w_mod, spec_diff, rvmin=-30, rvmax=30, drv=0.01, mode="lin", skipedge=True)
zz = pyasl.crosscorrRV(w_mod2, flux2, w_mod, flux, rvmin=-30, rvmax=30, drv=0.01, mode="lin", skipedge=True)


In [ ]:
plt.plot(z[0], z[1])
plt.show()

plt.plot(zz[0], zz[1])
plt.show()

In [ ]:
## AUTO CORRELATION OF observatoin
#wobs, fobs
mask = (wobs < 2119) & (wobs > 2113)
wobs2 = wobs[mask]
fobs2 = fobs[mask]


ac = pyasl.crosscorrRV(wobs2, fobs2, wobs, fobs, rvmin=-30, rvmax=30, drv=0.01, mode="lin", skipedge=True)

plt.plot(ac[0], ac[1])
plt.show()